# Expression Operators — MongoDB

Ce notebook est dédié aux **opérateurs d'expression MongoDB**.  
Il a été fait pour un travail en groupe.

Chaque membre peut compléter une section en ajoutant :
- Une explication simple
- Un exemple compréhensible par un non-technicien
- Une requête d’agrégation MongoDB illustrant l’opérateur

---

In [9]:
from pymongo import MongoClient
import pandas as pd

# Définition du port à utiliser [à changer en fonction la configuration de votre installation locale]
MONGODB_LOCAL_PATH = "mongodb://localhost:27018/"

# Définition de la base de données à utiliser
MONGODB_NAME = "test_db"

# Connexion
client = MongoClient(MONGODB_LOCAL_PATH)
db = client[MONGODB_NAME]

## Opérateur `$sum`

L’opérateur `$sum` permet d’additionner des valeurs.

---

In [10]:
#Code à ajouter

## Opérateur `$avg`

L’opérateur `$avg` calcule la moyenne de plusieurs valeurs numériques.

---

In [11]:
#Code à ajouter

## Opérateur `$min`

L’opérateur `$min` retourne la valeur la plus petite d’un groupe.

---

In [12]:
#Code à ajouter

## Opérateur `$max`

L’opérateur `$max` retourne la valeur la plus grande d’un groupe.

---

In [13]:
#Code à ajouter

## Opérateur `$first`

L’opérateur `$first` retourne le premier document ou la première valeur d’un groupe, selon l’ordre d’arrivée.

---

In [14]:
#Code à ajouter

## Opérateur `$last`

L’opérateur `$last` retourne le dernier document ou la dernière valeur d’un groupe, selon l’ordre d’arrivée.

---

In [15]:
#Code à ajouter

## Opérateur `$concat`

L’opérateur `$concat` est un opérateur d'aggrégation qui permet de fusionner plusieurs chaînes de caractères entre elles. Il s'invoque de la façon suivante:

`{ $concat: [ <expression1>, <expression2>, ... ] }`

---

Pour l'exemple, on va créer une collection de comptes bancaires.

In [ ]:
try:
    my_collection = db.create_collection("concat")
except:
    db.concat.drop()
    my_collection = db.create_collection("concat")

documents = [
    {"account": "A", "amount": "1890", "currency": "€"},
    {"account": "B", "amount": "2531", "currency": "£"},
    {"account": "C", "amount": "936", "currency": "$"},
    {"account": "D", "amount": "12534", "currency": "€"},
    {"account": "E", "amount": "9876", "currency": "$"},
]
result_many = my_collection.insert_many(documents)

On va crée un champ **info** qui concatène les champs **amount** et **currency** dans une phrase grâce à l'opérateur d'aggrégation `$concat`.

In [29]:
info = {
    "$addFields": {
        "info": {"$concat": ["Ce compte contient ","$amount", " ", "$currency", "."]}
    }
}
pd.DataFrame(my_collection.aggregate([info]).to_list())

,_id,account,amount,currency,info
0,6888e21b7747a8c5c9888151,A,1890,€,Ce compte contient 1890 €.
1,6888e21b7747a8c5c9888152,B,2531,£,Ce compte contient 2531 £.
2,6888e21b7747a8c5c9888153,C,936,$,Ce compte contient 936 $.
3,6888e21b7747a8c5c9888154,D,12534,€,Ce compte contient 12534 €.
4,6888e21b7747a8c5c9888155,E,9876,$,Ce compte contient 9876 $.


## Opérateur `$substrBytes`

L’opérateur `$substrBytes` permet d’extraire une sous-chaîne de caractères à partir d’une chaîne plus grande en indiquant le premier indice et le nombre de caractères (le nombre de Bytes) que l'on souhaite extraire. Il s'invoque de la façon suivante:

`{ $substrBytes: [ <string expression>, <byte index>, <byte count> ] }`

---

Pour l'exemple, on va créer une collection de comptes bancaires.

In [ ]:
try:
    my_collection = db.create_collection("substrBytes")
except:
    db.substrBytes.drop()
    my_collection = db.create_collection("substrBytes")

documents = [
    {"account": "A", "amount": "Ce compte contient  1890 €."},
    {"account": "B", "amount": "Ce compte contient  2531 €."},
    {"account": "C", "amount": "Ce compte contient   936 €."},
    {"account": "D", "amount": "Ce compte contient 12534 €."},
    {"account": "E", "amount": "Ce compte contient  9876 €."},
]
result_many = my_collection.insert_many(documents)

Le champ **amount** est une longue chaîne de caractères, on souhaite en extraire uniquement le montant grâce à l'opérateur d'aggrégation `$substrBytes`.

In [27]:
amounts_money = {
    "$addFields": {
        "money": {"$trim": {"input": {
            "$substrBytes": ["$amount", 19, 5]
            }
        }}
    }
}
pd.DataFrame(my_collection.aggregate([amounts_money]).to_list())

,_id,account,amount,money
0,6888dcc47747a8c5c988814c,A,Ce compte contient 1890 €.,1890
1,6888dcc47747a8c5c988814d,B,Ce compte contient 2531 €.,2531
2,6888dcc47747a8c5c988814e,C,Ce compte contient 936 €.,936
3,6888dcc47747a8c5c988814f,D,Ce compte contient 12534 €.,12534
4,6888dcc47747a8c5c9888150,E,Ce compte contient 9876 €.,9876


## Opérateur `$toInt`

L’opérateur `$toInt` permet de convertir une valeur en entier (integer). C'est un opérateur d'aggrégation, il s'invoque de la façon suivante.

`{$toInt: <expression>}`

---

Pour l'exemple, on va créer une collection de comptes bancaires.


In [ ]:
try:
    my_collection = db.create_collection("toInt")
except:
    db.toInt.drop()
    my_collection = db.create_collection("toInt")

documents = [
    {"account": "A", "amount": "1890"},
    {"account": "B", "amount": "2531"},
    {"account": "C", "amount": "936"},
    {"account": "D", "amount": "12534"},
    {"account": "E", "amount": "9876"},
]
result_many = my_collection.insert_many(documents)

Le champ **amount** étant une chaîne de caractères, on peut les convertir grâce à l'opérateur d'aggrégation `$toInt`.

In [23]:
amounts_money = {
    "$addFields": {
        "money": {"$toInt": "$amount"}
    }
}
pd.DataFrame(my_collection.aggregate([amounts_money]).to_list())

,_id,account,amount,money
0,6888db807747a8c5c9888147,A,1890,1890
1,6888db807747a8c5c9888148,B,2531,2531
2,6888db807747a8c5c9888149,C,936,936
3,6888db807747a8c5c988814a,D,12534,12534
4,6888db807747a8c5c988814b,E,9876,9876


## Opérateur `$toDouble`

L’opérateur `$toDouble` permet de convertir une valeur en nombre décimal (float). C'est un opérateur d'aggrégation, il s'invoque de la façon suivante.

`{$toDouble: <expression>}`

---

Pour l'exemple, on va créer une collection de compte bancaire.

In [20]:
try:
    my_collection = db.create_collection("toDouble")
except:
    db.toDouble.drop()
    my_collection = db.create_collection("toDouble")

documents = [
    {"account": "A", "amount": "1890.59"},
    {"account": "B", "amount": "2531.48"},
    {"account": "C", "amount": "936.10"},
    {"account": "D", "amount": "12534.21"},
    {"account": "E", "amount": "9876.54"},
]
result_many = my_collection.insert_many(documents)

Le champ **amount** étant une chaîne de caractères, on peut les convertir grâce à l'opérateur d'aggrégation `$toDouble`.

In [21]:
amounts_money = {
    "$addFields": {
        "money": {"$toDouble": "$amount"}
    }
}
pd.DataFrame(my_collection.aggregate([amounts_money]).to_list())

,_id,account,amount,money
0,6888daae7747a8c5c9888142,A,1890.59,1890.59
1,6888daae7747a8c5c9888143,B,2531.48,2531.48
2,6888daae7747a8c5c9888144,C,936.10,936.10
3,6888daae7747a8c5c9888145,D,12534.21,12534.21
4,6888daae7747a8c5c9888146,E,9876.54,9876.54


---

## Conclusion